In [2]:
import pypowsybl.network as pn
import pypowsybl
import jinja2
import pandas as pd
pd.set_option('mode.chained_assignment', None)

In [3]:
pd.__version__

'1.4.2'

In [4]:
n1 = pn.load('case1/case1.zip')

In [5]:
l1 = n1.get_lines()
l1.head(3)

,name,r,x,g1,b1,g2,b2,p1,q1,i1,p2,q2,i2,voltage_level1_id,voltage_level2_id,bus1_id,bus2_id,connected1,connected2
id,,,,,,,,,,,,,,,,,,,
L7-8-0,L7-8-0,0.85,7.20,0.0,0.000745,0.0,0.000745,0.0,0.0,0.0,0.0,0.0,0.0,VL2,VL8,VL2_1,VL8_0,True,True
L9-8-0,L9-8-0,1.19,10.08,0.0,0.001045,0.0,0.001045,0.0,0.0,0.0,0.0,0.0,0.0,VL3,VL8,VL3_1,VL8_0,True,True
L7-5-0,L7-5-0,3.20,16.10,0.0,0.001530,0.0,0.001530,0.0,0.0,0.0,0.0,0.0,0.0,VL2,VL5,VL2_1,VL5_0,True,True


In [6]:
n2 = pn.load('case2/case2.zip')
l2 = n2.get_lines()
l2.head(3)

,name,r,x,g1,b1,g2,b2,p1,q1,i1,p2,q2,i2,voltage_level1_id,voltage_level2_id,bus1_id,bus2_id,connected1,connected2
id,,,,,,,,,,,,,,,,,,,
L7-8-0,L7-8-0,3.14,1.59,0.0,0.000745,0.0,0.000745,0.0,0.0,0.0,0.0,0.0,0.0,VL2,VL8,VL2_1,VL8_0,True,True
L7-5-0,L7-5-0,3.20,16.10,0.0,0.001530,0.0,0.001530,0.0,0.0,0.0,0.0,0.0,0.0,VL2,VL5,VL2_1,VL5_0,True,True
L9-6-0,L9-6-0,3.90,17.00,0.0,0.001790,0.0,0.001790,0.0,0.0,0.0,0.0,0.0,0.0,VL3,VL6,VL3_1,VL6_0,True,True


In [7]:
cols = ['name', 'r', 'x', 'b1', 'b2', 'bus1_id', 'bus2_id']
l1 = l1[cols]
l2 = l2[cols]

In [8]:
# Compare only works on changes - not on add/removes
# lc = pd.DataFrame.compare(l1, l2) # After 1.5 use the kwarg result_names
# lc = lc.rename(columns={'self': 'orig', 'other': 'new'}, level=1)

In [9]:
# New Branches
mask_new = ~l2['name'].isin(l1['name'])
mask_delete = ~l1['name'].isin(l2['name'])

In [10]:
print("New Branches")
print(l2[mask_new])

New Branches
Empty DataFrame
Columns: [name, r, x, b1, b2, bus1_id, bus2_id]
Index: []


In [11]:
print("Deleted Branches")
print(l1[mask_delete])

Deleted Branches
          name     r      x        b1        b2 bus1_id bus2_id
id                                                             
L9-8-0  L9-8-0  1.19  10.08  0.001045  0.001045   VL3_1   VL8_0


In [12]:
# Might be bigO slow
l1_same = l1[l1['name'].isin(l2['name'])]
l2_same = l2[l2['name'].isin(l1['name'])]
lc = pd.DataFrame.compare(l1_same, l2_same, keep_shape=False, keep_equal=False) # After 1.5 use the kwarg result_names
lc = lc.rename(columns={'self': 'orig', 'other': 'new'}, level=1)
changed_ids = lc.index
print(changed_ids)

Index(['L7-8-0'], dtype='object', name='id')


In [13]:
print("Changed Branches")
l1_same = l1_same.loc[changed_ids]
l2_same = l2_same.loc[changed_ids]
l1_same['source'] = 'orig'
l2_same['source'] = 'new'

l1_same
mm = pd.concat([l1_same, l2_same])
mm = mm.set_index('source', append=True)
# mm.sort_index('source')
mm

Changed Branches


name     r     x        b1        b2 bus1_id bus2_id
id     source                                                        
L7-8-0 orig    L7-8-0  0.85  7.20  0.000745  0.000745   VL2_1   VL8_0
       new     L7-8-0  3.14  1.59  0.000745  0.000745   VL2_1   VL8_0

## Draw changes

In [14]:
busses_touched = []
busses_touched.extend(l1[mask_delete]['bus1_id'].tolist())
busses_touched.extend(l1[mask_delete]['bus2_id'].tolist())
busses_touched.extend(l2[mask_new]['bus1_id'].tolist())
busses_touched.extend(l2[mask_new]['bus2_id'].tolist())
busses_touched.extend(mm['bus1_id'].tolist())
busses_touched.extend(mm['bus2_id'].tolist())
print(f"busses_touched (duplicates): {busses_touched}")
busses_touched = pd.Series(busses_touched).unique().tolist()
print(f"busses_touched (unique): {busses_touched}")

busses_touched (duplicates): ['VL3_1', 'VL8_0', 'VL2_1', 'VL2_1', 'VL8_0', 'VL8_0']
busses_touched (unique): ['VL3_1', 'VL8_0', 'VL2_1']


In [15]:
# n1_voltage_levels.head(3)
# n1_busses
# n1_substations.head(3)

In [16]:
n1_busses = n1.get_buses(all_attributes=True)
n1_voltage_levels = n1.get_voltage_levels(all_attributes=True)
n1_substations = n1.get_substations(all_attributes=True)

In [17]:
n1_drawme_vls = n1_voltage_levels.loc[n1_busses.loc[busses_touched]['voltage_level_id']]
n1_drawme_substation = n1_substations.loc[n1_drawme_vls['substation_id']]

In [18]:
case1_station_changes = []
for substation in n1_drawme_substation.reset_index().to_dict(orient='records'):
    dd = {}
    dd['substation_id'] = substation['id']
    dd['substation_name'] = substation['name']
    svg = 'case1_{}.svg'.format(substation['name'])
    dd['svg'] = svg
    case1_station_changes.append(dd.copy())
case1_station_changes

[{'substation_id': 'S3', 'substation_name': 'S3', 'svg': 'case1_S3.svg'},
 {'substation_id': 'S8', 'substation_name': 'S8', 'svg': 'case1_S8.svg'},
 {'substation_id': 'S2', 'substation_name': 'S2', 'svg': 'case1_S2.svg'}]

In [19]:
for sc in case1_station_changes:
    n1.write_single_line_diagram_svg(sc['substation_id'], sc['svg'])
case1_area_changes = 'case1_area_changes.svg'    
n1.write_network_area_diagram_svg(case1_area_changes, n1_drawme_vls.index.tolist())

In [20]:
n2_busses = n2.get_buses(all_attributes=True)
n2_voltage_levels = n2.get_voltage_levels(all_attributes=True)
n2_substations = n2.get_substations(all_attributes=True)

In [21]:
n2_busses.head(1)

,name,v_mag,v_angle,connected_component,synchronous_component,voltage_level_id,fictitious
id,,,,,,,
VL1_0,VL1_0,104.0,0.0,0,0,VL1,False


In [22]:
n2_drawme_vls = n2_voltage_levels.loc[n2_busses.loc[busses_touched]['voltage_level_id']]
n2_drawme_vls

,name,substation_id,nominal_v,high_voltage_limit,low_voltage_limit,fictitious,topology_kind
id,,,,,,,
VL3,VL3,S3,100.0,NaN,NaN,False,BUS_BREAKER
VL8,VL8,S8,100.0,NaN,NaN,False,BUS_BREAKER
VL2,VL2,S2,100.0,NaN,NaN,False,BUS_BREAKER


In [23]:
n2_drawme_substation = n2_substations.loc[n2_drawme_vls['substation_id']]
n2_drawme_substation

,name,TSO,geo_tags,country,fictitious,CGMES.regionName,CGMES.subRegionId,CGMES.regionId
id,,,,,,,,
S3,S3,,default region,,False,default region,09655630-f880-43ab-a104-3b37624d45fa,195fe606-d48f-4d53-8279-e7c0b317ef4b
S8,S8,,default region,,False,default region,09655630-f880-43ab-a104-3b37624d45fa,195fe606-d48f-4d53-8279-e7c0b317ef4b
S2,S2,,default region,,False,default region,09655630-f880-43ab-a104-3b37624d45fa,195fe606-d48f-4d53-8279-e7c0b317ef4b


In [24]:
case2_station_changes = []
for substation in n2_drawme_substation.reset_index().to_dict(orient='records'):
    dd = {}
    dd['substation_id'] = substation['id']
    dd['substation_name'] = substation['name']
    svg = 'case2_{}.svg'.format(substation['name'])
    dd['svg'] = svg
    case2_station_changes.append(dd.copy())
case2_station_changes

[{'substation_id': 'S3', 'substation_name': 'S3', 'svg': 'case2_S3.svg'},
 {'substation_id': 'S8', 'substation_name': 'S8', 'svg': 'case2_S8.svg'},
 {'substation_id': 'S2', 'substation_name': 'S2', 'svg': 'case2_S2.svg'}]

In [25]:
for sc in case2_station_changes:
    n2.write_single_line_diagram_svg(sc['substation_id'], sc['svg'])
case2_area_changes = 'case2_area_changes.svg'
n2.write_network_area_diagram_svg(case2_area_changes, n2_drawme_vls.index.tolist())

In [ ]:
station_changes = 

In [26]:
with open('template.html', 'r') as f:
    template_string = f.read()
template = jinja2.Template(template_string)
context = {'case1_area_changes': case1_area_changes, 'case2_area_changes': case2_area_changes,
          'case1_station_changes': case1_station_changes, 'case2_station_changes': case2_station_changes}
html = template.render(context)

with open('compare_report.html', 'w') as f: 
    f.write(html)